# Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details. You can download the required DB from the shared dropbox or from blackboard

In [48]:
# import packages and libraries
import sqlite3
import nltk
# nltk.download('punkt')
import random
import numpy as np
from collections import Counter, defaultdict

from string import punctuation
from nltk.tokenize import sent_tokenize
import os

# Feel free to include your text patterns functions
# from text_functions_solutions import clean_tokenize, get_patterns

In [25]:
# create connection to database
convention_db = sqlite3.connect("2020_Conventions.db")
convention_cur = convention_db.cursor()

## 1. Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" exercise. First, we'll pull in the text 
for each party and prepare it for use in Naive Bayes. 

In [26]:
# Query to pull data and exclude 'Other' party
query = '''
    SELECT text, party
    FROM conventions
    WHERE party != 'Other'
'''

# Execute the query and fetch all the data
query_results = convention_cur.execute(query).fetchall()

# Initialize the list to store results
convention_data = [[row[0], row[1]] for row in query_results]

# Close the database connection
convention_db.close()

# Print only a few records from the convention_data for verification
print(convention_data[:5])  

[['Skip to content The Company Careers Press Freelancers Blog × Services Transcription Captions Foreign Subtitles Translation Freelancers About Contact Login « Return to Transcript Library home  Transcript Categories  All Transcripts 2020 Election Transcripts Classic Speech Transcripts Congressional Testimony & Hearing Transcripts Debate Transcripts Donald Trump Transcripts Entertainment Transcripts Financial Transcripts Interview Transcripts Political Transcripts Press Conference Transcripts Speech Transcripts Sports Transcripts Technology Transcripts Aug 21, 2020 2020 Democratic National Convention (DNC) Night 4 Transcript Rev  ›  Blog  ›  Transcripts  › 2020 Election Transcripts  ›  2020 Democratic National Convention (DNC) Night 4 Transcript Night 4 of the 2020 Democratic National Convention (DNC) on August 20. Read the full transcript of the event here. Transcribe Your Own Content  Try Rev for free  and save time transcribing, captioning, and subtitling.', 'Democratic'], ['I’m her

Let's look at some random entries and see if they look right. 

In [27]:
# Print random entries from database
random.choices(convention_data,k=5)

[['In a democracy, we do not elect saviors. We cast our ballots for those who see our struggles and pledge to serve, who hear our dreams and work to make them real. Who defend our way of life by protecting our right to vote. Faced with a president of cowardice, Joe Biden is a man of proven courage. He will restore our moral compass by confronting our challenges, not by hiding from them or undermining our elections to keep his job. In a time of voter suppression at home and authoritarians abroad, Joe Biden will be a champion for free and fair elections, for a public health system that keeps us safe, for an economy that we build back better than before and for accountability and integrity in our system of justice. We stand with Joe Biden because this isn’t just about defeating Donald Trump. We are in this to win for America, so let’s get it done.',
  'Democratic'],
 ['We will never, ever sign bad trade deals. America first, again, America first.',
  'Republican'],
 ['He’s included my fam

It'll be useful for us to have a large sample size than 2024 affords, since those speeches tend to be long and contiguous. Let's make a new list-of-lists called `conv_sent_data`. Instead of each first entry in the sublists being an entire speech, make each first entry just a sentence from the speech. Feel free to use NLTK's `sent_tokenize` [function](https://www.nltk.org/api/nltk.tokenize.sent_tokenize.html). 

In [54]:
# Load the Punkt tokenizer correctly using nltk.data.load
punkt_path = r'C:\Users\marvi\AppData\Roaming\nltk_data\tokenizers\punkt\english.pickle'

# Check if the punkt file exists
if os.path.exists(punkt_path):
    tokenizer = nltk.data.load(f'file:{punkt_path}')  # Properly load the pickle file with nltk
else:
    raise FileNotFoundError(f"Punkt tokenizer not found at: {punkt_path}")

# Initialize the list to store individual sentences
conv_sent_data = []

# Process each speech in convention_data and split it into sentences
for row in convention_data:
    speech_text, party = row
    # Tokenize the speech text into sentences using the manually loaded tokenizer
    sentences = tokenizer.tokenize(speech_text)  # Use the manually loaded tokenizer
    # Append each sentence with the respective party label to conv_sent_data
    for sentence in sentences:
        conv_sent_data.append([sentence, party])

# Print only a few records from conv_sent_data for verification
print(conv_sent_data[:5])  # Prints the first 5 records


UnpicklingError: global 'copy_reg._reconstructor' is forbidden

Again, let's look at some random entries. 

In [ ]:
random.choices(conv_sent_data,k=5)

Now it's time for our final cleaning before modeling. Go through `conv_sent_data` and take the following steps: 

1. Tokenize on whitespace
1. Remove punctuation
1. Remove tokens that fail the `isalpha` test
1. Remove stopwords
1. Casefold to lowercase
1. Join the remaining tokens into a string


In [ ]:
clean_conv_sent_data = [] # list of tuples (sentence, party), with sentence cleaned

for idx, sent_party in enumerate(conv_sent_data) :
    pass # your code here

random.choices(clean_conv_sent_data,k=5)

[('maybe one day catch', 'Republican'),
 ('change within change made last', 'Republican'),
 ('whoo', 'Republican'),
 ('want talk man know', 'Republican'),
 ('lucky', 'Republican')]

If that looks good, let's make our function to turn these into features. First we need to build our list of candidate words. I started my exploration at a cutoff of 5. 

In [ ]:
word_cutoff = 5

tokens = [w for t, p in clean_conv_sent_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 1246 as features in the model.


In [ ]:
def conv_features(text,fw) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    
    # Your code here
    
    ret_dict = dict()
    
    return(ret_dict)

In [ ]:
assert(len(feature_words)>0)
assert(conv_features("obama was the president",feature_words)==
       {'obama':True,'president':True})
assert(conv_features("some people in america are citizens",feature_words)==
                     {'people':True,'america':True,"citizens":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [ ]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [ ]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [ ]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

In [ ]:
classifier.show_most_informative_features(25)

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

_Your observations to come._



## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [ ]:
cong_db = sqlite3.connect("congressional_twitter_data.db")
cong_cur = cong_db.cursor()

In [ ]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [ ]:
tweet_data = []

# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.


There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [ ]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

In [ ]:

for tweet, party in tweet_data_sample :
    estimated_party = '??' # you need to fill this in.
    # Fill in the right-hand side above with code that estimates the actual party
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")
    

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [ ]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
   
    # get the estimated party
    estimated_party = "Gotta fill this in"
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [ ]:
results

### Reflections

_Write a little about what you see in the results_ 